<h1>Chapter 7 - langchain-and-langgraph-basic</h1>

- Agent 最重要的两个道路：

- Prompt Engineering Based Agent System
    - 今天我们讲的就是这个 系统怎么构建；
- RL 强化学习优化单个模型的能力上限
    - 推荐赵老师的 《强化学习数学原理》


## 1. 理解 Langchain Basic

In [11]:
# %%capture
# !pip install langchain
"""
langchain                      0.3.25
langchain-core                 0.3.65
langchain-openai               0.3.23
langchain-text-splitters       0.3.8
langgraph                      0.4.8
langgraph-checkpoint           2.0.26
langgraph-prebuilt             0.2.2
langgraph-sdk                  0.1.70
langsmith                      0.3.45
"""

'\nlangchain                      0.3.25\nlangchain-core                 0.3.65\nlangchain-openai               0.3.23\nlangchain-text-splitters       0.3.8\nlanggraph                      0.4.8\nlanggraph-checkpoint           2.0.26\nlanggraph-prebuilt             0.2.2\nlanggraph-sdk                  0.1.70\nlangsmith                      0.3.45\n'

In [185]:
from dotenv import load_dotenv
import os

load_dotenv(".env")


True

In [188]:
BASE_URL = "https://api.deepseek.com"
API_KEY = os.environ.get("API_KEY")

deepseek_chat_model = "deepseek-chat"
# deepseek_reason_model = "deepseek-reasoner"

1. langchain, init model;  使用 invoke， 以及 prompt template
2. langgraph memory
3. langgraph 的 create_react_agent

### 1.1 方式1 openai sdk
通过 OpenAI sdk

In [191]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate

llm1 = ChatOpenAI(model=deepseek_chat_model, api_key=API_KEY, base_url=BASE_URL)

llm1.invoke([HumanMessage("帮我写一个关于 电脑的笑话")])

AIMessage(content='好的！这里有一个关于电脑的笑话：\n\n---\n\n**笑话：电脑的抱怨**\n\n有一天，一个人对他的电脑说：“你真是太慢了，简直像蜗牛一样！”  \n电脑冷冷地回复：“嘿，至少我从不‘死机’……哦，等等。”  \n（屏幕突然蓝屏）  \n\n---\n\n希望这个笑话能让你笑一笑！如果需要其他类型的笑话，随时告诉我哦！ 😄', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 11, 'total_tokens': 93, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 11}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': 'bf2db262-2625-4eaf-b579-88ec3921ef91', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--90e2e2b7-4c29-45d3-95af-61ab652d106c-0', usage_metadata={'input_tokens': 11, 'output_tokens': 82, 'total_tokens': 93, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

### 1.2 方式2 deepseek
使用 deepseek sdk

In [18]:
# !pip install -U langchain-deepseek

In [193]:
# 其他

from langchain_deepseek import ChatDeepSeek

llm2 = ChatDeepSeek(model=deepseek_chat_model, api_key=API_KEY)
messages = [HumanMessage("你是谁？")]
print(llm2.invoke(messages).content)

我是DeepSeek Chat，由深度求索公司（DeepSeek）创造的智能AI助手！🤖✨ 我的使命是帮助你解答问题、提供信息、陪你聊天，甚至帮你处理各种文本和文件内容。无论是学习、工作，还是日常生活中的疑问，我都会尽力帮你解决！😊  

有什么我可以帮你的吗？


### 1.3 使用 统一的 方式

In [194]:
# pip install -U langchain-deepseek
from langchain.chat_models import init_chat_model

llm3 = init_chat_model(
    deepseek_chat_model, # deepseek-chat
    api_key=API_KEY,
)
print(llm3.invoke(messages).content)


我是DeepSeek Chat，由深度求索公司（DeepSeek）创造的智能AI助手！✨ 我的使命是帮助你解答问题、提供建议、陪你聊天，还能处理各种文本、文档等内容。无论是学习、工作，还是日常生活中的疑问，我都会尽力帮你解决！😊  

有什么我可以帮你的吗？


# 2. 了解 LangChain Chain

### 2.1 ChatAgent Using LangChain

In [195]:
from langchain_core.prompts import ChatPromptTemplate

In [196]:
prompt_template = ChatPromptTemplate(
    [
        ("system", "you are a helpful assistant"),
        ("human", "帮我写一个和 {topic} 相关的笑话"),
    ]
)

In [197]:
prompt_template.input_variables

['topic']

In [ ]:
# llm.invoke(messages)
# llm.invoke({"topic": "xxxxx"})

In [198]:
prompt_template.invoke({"topic": "cat"}).to_messages()

[SystemMessage(content='you are a helpful assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='帮我写一个和 cat 相关的笑话', additional_kwargs={}, response_metadata={})]

In [199]:
# langchain 的 Chain 是什么意思？
# A | B | C | D
basic_chain = prompt_template | llm2

In [200]:
# Use the chain
ans = basic_chain.invoke(
    {
        "topic": "cat",
    }
)
print(ans.content)

当然！这里有一个和猫有关的笑话：

---

**笑话：**

有一天，一只猫走进了一家酒吧。  
酒保看着它问：“你想喝点什么？”  
猫想了想说：“给我来一杯‘喵喵白葡萄酒’。”  
酒保好奇地问：“这名字挺特别，为什么叫这个？”  
猫淡定地回答：“因为每次我喝完，都会‘喵～’地唱起来！”  

---

希望这个笑话能让你开心！😺


### 2.2 结构化输出

In [ ]:
from pydantic import BaseModel, Field

class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")


# 创建 chain
joke_chain = prompt_template | llm2.with_structured_output(Joke)

# 调用 chain，结构化输出
joke_chain.invoke({"topic": "cat"})

# 缺点： FunctionCall 能力的模型才能使用；

Joke(setup='Why did the cat sit on the computer?', punchline='Because it wanted to keep an eye on the mouse!')

In [202]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser


class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")


# 创建包含输出格式要求的提示模板
prompt_template = ChatPromptTemplate(
    [
        (
            "system",
            """你是一个幽默助手。请按照以下JSON格式返回笑话:
        {{
            "setup": "笑话的铺垫",
            "punchline": "笑话的包袱"
        }}
        只返回JSON格式,不要其他内容。""",
        ),
        ("human", "帮我写一个和 {topic} 相关的笑话"),
    ]
)

# 创建解析器
parser = JsonOutputParser(pydantic_object=Joke)

# 创建 chain
joke_chain = prompt_template | llm2 | parser

# 调用 chain
response = joke_chain.invoke({"topic": "cat"})
print(response)

{'setup': '为什么猫咪不喜欢玩扑克？', 'punchline': '因为遇到黑猫它们总是输！'}


## 总结一下
1. 统一的用 init_chat_model / ChatXxxModel 初始化一个 chat model
2. 统一的用 invoke(messages)  or invoke({"topic": "xxxx"})
3. 我们可以用 PromptTemplate 构建结构化的 模板
4. （不重点），结构化输出

# 3. LangGraph Memory

In [204]:
output = llm2.invoke(messages)

# memory 
# message 以及 output ， 两者要我们自己加起来；

In [ ]:
# class MessagesState(TypedDict):
#     messages: Annotated[list[AnyMessage], add_messages]

# {
#     "messages": []
# }

In [206]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = llm2.invoke(state["messages"])
    return {"messages": response}


workflow.add_node("node1", call_model)

# Define the (single) node in the graph
workflow.add_edge(START, "node1")

# Add memory
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)


In [207]:
config = {"configurable": {"thread_id": "abc123"}}
query = "Hi! I'm chaofa."

input_messages = [HumanMessage(query)]
output = graph.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Chaofa! 👋 It's great to meet you. How can I assist you today? Whether you have questions, need recommendations, or just want to chat, I'm here to help! 😊  

Let me know what's on your mind—maybe a fun fact, a tricky problem, or even your favorite hobby?


In [208]:
output

{'messages': [HumanMessage(content="Hi! I'm chaofa.", additional_kwargs={}, response_metadata={}, id='5c5441e2-b85f-4522-8a5b-e105ed75c06f'),
  AIMessage(content="Hi Chaofa! 👋 It's great to meet you. How can I assist you today? Whether you have questions, need recommendations, or just want to chat, I'm here to help! 😊  \n\nLet me know what's on your mind—maybe a fun fact, a tricky problem, or even your favorite hobby?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 10, 'total_tokens': 77, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 10}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': 'f5d1c125-6ec3-4a17-9b45-4d2a2f693f7b', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--c5f75e2e-c1a2-47ff-b437-c9bb5d36bcae-0', usage_metadata={'input_toke

In [ ]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = graph.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is **Chaofa**! 🌟 (I remember because you've mentioned it twice—let me know if you'd prefer a nickname!)  

What’s on your mind, Chaofa? 😊


In [209]:
config = {"configurable": {"thread_id": "abc345"}}

query = "What's my name?"

input_messages = [HumanMessage(query)]
output = graph.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don’t have access to personal information about you unless you share it with me. If you’d like, you can tell me your name, and I’ll be happy to use it in our conversation! 😊


# 4. Agents

## 4.1 Single Agent(With Tool)

In [211]:
# function call 使用和 訓練
# https://www.bilibili.com/video/BV1cqdPY8EzT/

In [212]:
from langgraph.prebuilt import create_react_agent

In [213]:
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [214]:
agent = create_react_agent(
    model=ChatDeepSeek(model=deepseek_chat_model, api_key=API_KEY),
    tools=[multiply, add],
)

In [215]:
output = agent.invoke({"messages": [HumanMessage("帮我计算 1 + 2")]})

In [216]:
for item in output["messages"]:
    item.pretty_print()

================================ Human Message =================================

帮我计算 1 + 2
================================== Ai Message ==================================
Tool Calls:
  add (call_0_c1d0d208-ae81-4ded-a7ff-4bdb69b74666)
 Call ID: call_0_c1d0d208-ae81-4ded-a7ff-4bdb69b74666
  Args:
    a: 1
    b: 2
================================= Tool Message =================================
Name: add

3
================================== Ai Message ==================================

1 + 2 = 3


In [125]:
multi_output = agent.invoke({"messages": [HumanMessage("帮我计算 1 * 2")]})

for item in multi_output["messages"]:
    item.pretty_print()

================================ Human Message =================================

帮我计算 1 * 2
================================== Ai Message ==================================
Tool Calls:
  multiply (call_0_893cc7fc-8ac8-4012-a6c4-fff050050543)
 Call ID: call_0_893cc7fc-8ac8-4012-a6c4-fff050050543
  Args:
    a: 1
    b: 2
================================= Tool Message =================================
Name: multiply

2
================================== Ai Message ==================================

1 乘以 2 的结果是 2。


## 4.2 DeepResearch

In [144]:
# !pip install langchain_community
# !!pip install -U duckduckgo-search
# !!pip install -U langchain-tavily

### dr 概念理解
单一目标执行原则；

1. planning agent 做规划，要搜索什么东西；
2. search agent，根据 plan 调用 Search tool 检索结果；（可以循环的搜索）
3. report agent，只需要写报告；

In [147]:
from langchain_tavily import TavilySearch

search_tool = TavilySearch(
    max_results=5, topic="general", tavily_api_key=os.environ.get("TAVILY_KEY")
)
search_tool.invoke("chaofa用代码打点酱油")

{'query': 'chaofa用代码打点酱油',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'bbruceyuan (Chaofa Yuan) - GitHub',
   'url': 'https://github.com/bbruceyuan',
   'content': '博客：https://bruceyuan.com 公众号👏：chaofa用代码打点酱油. bbruceyuan has 25 repositories available. Follow their code on GitHub.',
   'score': 0.76554126,
   'raw_content': None},
  {'title': 'chaofa用代码打点酱油 | Notion',
   'url': 'https://chaofa.notion.site/11a569b3ecce49b2826d679f5e2fdb54?v=1550d5d337e68005a0d5000c1d731b37',
   'content': 'hi，我是 chaofa，目前是个大模型算法工程师，对于搜索、广告也有我一定的了解~ 这是我的 notion blog. 记录平常的一些学习记录，主要是 paper阅读方面的内容~ 此外会记录一些公开的创作计划; 个人中文博客： chaofa用代码打点酱油 （推荐）',
   'score': 0.6506676,
   'raw_content': None},
  {'title': 'chaofa用代码打点酱油的动态 - 哔哩哔哩',
   'url': 'https://www.bilibili.com/opus/987060205629997076',
   'content': 'PyTorch手写多头注意力（Multi-Head Self-Attention）-- Self Attention 四重境界 part2（面试常考）',
   'score': 0.52962893,
   'raw_content': None},
  {'title': 'chaofa用代码打点酱油 - YouTube',

In [162]:
# from langchain_core.messages import HumanMessage, SystemMessage
# from langgraph.prebuilt import create_react_agent
# from typing import List
# from pprint import pprint
# from langgraph.types import Send

# # Create plan agent
# plan_prompt = """You are a research planner. Break down the research task into specific steps.
# Format: List 1-3 key questions to answer."""

# class QueryAndReason(BaseModel):
#     query: str = Field(..., description="检索的 Query，需要符合搜索习惯")
#     reason: str = Field(..., description="为什么要检索这个 query")

# # Define the structured output format for plan agent
# class ResearchPlan(BaseModel):
#     """Research plan with key questions"""

#     questions: List[QueryAndReason] = Field(
#         description="List of 1-3 key research questions to investigate.", max_items=3
#     )


# plan_agent = ChatDeepSeek(
#     model=deepseek_chat_model, api_key=API_KEY
# ).with_structured_output(ResearchPlan)


# pprint(plan_agent.invoke([HumanMessage("write an description about andrej karthy")]))

In [217]:
from pydantic import BaseModel, Field
from typing import List
from langgraph.graph import START, END

# Define the structured output format for plan agent
class ResearchPlan(BaseModel):
    """Research plan with key questions"""
    questions: List[str] = Field(
        description="List of 1-3 key research questions to investigate",
        max_items=3
    )

# planner agent 输出 json / markdown （ - [] 第一条要做的事情 -[ ] 第二条要做的事情）
# Create plan agent with structured output
plan_prompt = """You are a research planner. Break down the research task into specific steps.
Your response must be a JSON object with a 'questions' field containing 1-3 key research questions.

Example format:
{
    "questions": [
        "What is X?",
        "How does Y work?",
        "What are the implications of Z?"
    ]
}

Only return the JSON object, no other text."""

plan_agent = create_react_agent(
    model=ChatDeepSeek(model=deepseek_chat_model, api_key=API_KEY),
    tools=[],
    prompt=plan_prompt
)
search_agent = create_react_agent(
    model=ChatDeepSeek(model=deepseek_chat_model, api_key=API_KEY),
    tools=[search_tool],
    prompt="you are a helpful assistant. you can search result in search_tool. 当你获得检索结果之后，输出“检索完毕”，必要额外输出任何东西。",
)

report_agent = create_react_agent(
    model=ChatDeepSeek(model=deepseek_chat_model, api_key=API_KEY),
    tools=[],
    prompt="你是一个擅长写作的人。 你可以根据用户的问题，以及检索的结果，生成一个最终的报告。"
)

# plan -> search -> report 

# Create state graph with custom state
class ResearchState(MessagesState):
    """Custom state that includes research plan"""
    plan: ResearchPlan | None = None

# Update workflow with new state
workflow = StateGraph(ResearchState)


# Add nodes and edges with state transformations
workflow.add_node("plan_agent", plan_agent)
workflow.add_node("search_agent", search_agent)
workflow.add_node("report_agent", report_agent)

workflow.add_edge(START, "plan_agent")
workflow.add_edge("plan_agent", "search_agent")
workflow.add_edge("search_agent", "report_agent")
workflow.add_edge("report_agent", END)

# Compile workflow
app = workflow.compile()

In [173]:
output = app.invoke(
    {"messages": [
        HumanMessage("write an description about andrej karthy")
    ]}
)

In [174]:
output

{'messages': [HumanMessage(content='write an description about andrej karthy', additional_kwargs={}, response_metadata={}, id='6ec66ea6-15f4-4ee5-a70a-8354000a4acc'),
  AIMessage(content='```json\n{\n    "questions": [\n        "Who is Andrej Karpathy and what are his notable contributions in the field of AI and machine learning?",\n        "What roles has Andrej Karpathy held at companies like OpenAI and Tesla?",\n        "What are some of Andrej Karpathy\'s most influential projects or publications?"\n    ]\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 94, 'total_tokens': 169, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 64}, 'prompt_cache_hit_tokens': 64, 'prompt_cache_miss_tokens': 30}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': '00df10ab-adc5-4a2d-bad6-67a1391d03ce', 'service_tier': None, 'finish_reason': 

In [175]:
for item in output["messages"]:
    item.pretty_print()

================================ Human Message =================================

write an description about andrej karthy
================================== Ai Message ==================================

```json
{
    "questions": [
        "Who is Andrej Karpathy and what are his notable contributions in the field of AI and machine learning?",
        "What roles has Andrej Karpathy held at companies like OpenAI and Tesla?",
        "What are some of Andrej Karpathy's most influential projects or publications?"
    ]
}
```
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_0_9f568643-fc2c-4b87-83f8-24e746141279)
 Call ID: call_0_9f568643-fc2c-4b87-83f8-24e746141279
  Args:
    query: Who is Andrej Karpathy and what are his notable contributions in the field of AI and machine learning?
  tavily_search (call_1_c86e7717-4d5b-4ba0-a4d7-aea74be2848f)
 Call ID: call_1_c86e7717-4d5b-4ba0-a4d7-aea74be2848f
  Args:
    query: Wha

In [170]:
events = app.stream(
    {"messages": [{"role": "user", "content": "帮我写一个 chaofa 的个人生平"}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

帮我写一个 chaofa 的个人生平
================================== Ai Message ==================================

```json
{
    "questions": [
        "Who is Chaofa and what are their notable achievements?",
        "What are the key events in Chaofa's life?",
        "How has Chaofa contributed to their field or society?"
    ]
}
```
================================== Ai Message ==================================

以下是根据搜索结果整理的关于“Chaofa”的个人生平信息：

---

### **Chaofa 的生平与成就**

1. **Zhang Chaofa（张朝发）**  
   - **背景**：张朝发是中国清朝时期的一位将领，活跃于鸦片战争期间。  
   - **关键事件**：  
     - 在鸦片战争中，张朝发率领军队与英军作战，并在战斗中表现出色。  
     - 在一次战斗中，他利用敌军船只漏水的机会追击并取得胜利，因此被连续三次晋升，最终成为总兵（高级将领）。  
     - 然而，在定海失守后，朝廷调查责任时，张朝发被错误指控为战败的罪魁祸首，最终被处决。  
   - **贡献**：他的英勇作战和牺牲精神在鸦片战争中留下了深刻的印记。

2. **Hmong ChaoFa（苗族王国）**  
   - **背景**：Hmong ChaoFa 是一个致力于为老挝苗族人民争取自决权的组织。  
   - **关键事件**：  
     - 1998年，Hmong ChaoFa 通过了宪法，并开始为苗族人民的权利和自由奋斗。  
     - 该组织的领导人因其在冲突解决和受压迫人民权

In [183]:
from pydantic import BaseModel, Field


class A(BaseModel):
    x: str = Field(default="123", description="this is for x")

ins = A()
ins.model_dump()

{'x': '123'}